# Finding Trott Constants

> A Trott Constant is "a real number whose decimal digits are equal to the terms of its continued fraction."

So our first step is to create continued fraction calculation to an arbitrary number of decimal places (we can set how precise we wish to be). 

In [2]:
from mpmath import mp, mpf


def count_digits(cf_list):
    """A Function to count amount of digits in a list of numbers
    We will use this to figure out the amount of precision we want"""
    total = 0
    for num in cf_list:
        while num > 0: # note: numbers are guaranteed to be positive
            num = num // 10
            total = total + 1
    return total

In [3]:
a = [100, 5, 7, 99]
print(count_digits(a))
print(a)

7
[100, 5, 7, 99]


## Continued Fraction to Real Number conversion

In [4]:
def cf_to_real(cf_list, precision):
    with mp.workdps(precision):
        curr = mpf(0) # 0 is previous term
        for item in cf_list[::-1]: # loop through list backwards
            curr = 1 / (item + curr)
        return curr

In [5]:
cf = [3, 29, 5, 7]

cf_to_real(cf, count_digits(cf) + 1)

mpf('0.32957041263580322')

## Continued Fraction to Expected Trott Constant approximation

In [6]:
def cf_to_trott_approx(cf_list):
    with mp.workdps(count_digits(cf_list)+5):
        return mpf("0.{}".format("".join(map(str,cf_list))))

In [7]:
cf_to_trott_approx(cf)

mpf('0.32957000000169501')

## Check if CF form has equal real and expected values

The best way to check if the two numbers are equal is through the `almosteq` method which takes in both values and an epsilon which the difference of the two values have to be lower than to be considered "almost equal."

In [44]:
def valid_trott(cf_list):
    with mp.workdps(count_digits(cf_list)+1):
        dc = count_digits(cf_list)
        return mp.almosteq(cf_to_real(cf_list, dc + 1), cf_to_trott_approx(cf_list), 9/(10**(dc+1)))

In [46]:
cf = [3,29,5,7]
print(valid_trott(cf))

cf = [3,29,5,8]
print(valid_trott(cf))

print(valid_trott([3]))

True
False
True


## Testing for any possible Trott constant stems 3 items long

This is the general structure of what we wish to do to check if there are n-item long Trott constants. However doing this many nested for-loops gets tedious and it is actually not possible to general to an nth degree so we must use a recursive technique known as backtracking.

In [50]:
ITEM_MAX = 1000000

# for-loop solution (NOT backtracking)
for i in range(1, ITEM_MAX):
    a = [i]
    if valid_trott(a):
        print(a)
        for j in range(1, ITEM_MAX):
            a = [i,j]
            if valid_trott(a):
                print(a)
                for k in range(1, ITEM_MAX):
                    a=[i,j,k]
                    if valid_trott(a):
                        print(a)

[3]
[3, 29]
[3, 29, 5]
[3, 29, 6]
[3, 29, 54]
[3, 29, 55]
[3, 29, 545]
[3, 29, 546]
[3, 29, 5454]
[3, 29, 5455]
[3, 29, 54545]
[3, 29, 54546]
[3, 29, 545454]
[3, 29, 545455]
[3, 30]
[3, 330]
[3, 3330]
[3, 33330]
[3, 333330]
[10]


## Backtracking Solution
> backtracking is the best way to do a brute force for all permutations in larger cases.

Here we will take what we do in each of the for-loops in the code above and call this function recursively. I will set more limiting restrictions on this version as the above code actually does take a bit of time to run just for 3 items of in the range of `[1, 1000000]`.

In [74]:
def trott_backtracking(current_cf, count, max_items, max_size, file=None):
    if count < max_items:
        valid = []
        for a in range(1, max_size):
            new_cf = current_cf + [a]
            if valid_trott(new_cf):
                valid.append(new_cf)
            else:
                # This branch simply doesn't work
                print("INVALID: {}".format(str(new_cf)),file=file)
        if not valid:
            # This branch is dead -- it started out valid but had nowhere to go.
            print("DEAD: {}".format(str(current_cf)),file=file)
        else:
            for v in valid:
                # These branches are still alive
                print("VALID: {}".format(str(v)), file=file)
                trott_backtracking(v, count+1, max_items, max_size, file=file)

In [76]:
with open("output1.txt", "w+") as out:
    trott_backtracking([], 0, 1000, 50, out)

Since the output was long I moved it to another file but it can be seen in `output1.txt`.

Now I just need to run this for much larger possible values.

In [ ]:
with open("output2.txt", "w+") as out:
    trott_backtracking([], 0, 100, 1000, out)